In [1]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)

In [57]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [64]:
metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,24), test_inputs, 24, y_scaler)
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAPE')
    mae = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/global/baseline.csv')

100% |########################################################################|


In [ ]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

#FULL_LSTMIMO = tf.keras.models.Sequential([
#    # Shape [batch, time, features] => [batch, time, lstm_units]
#    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
#    # Shape => [batch, time, features]
#    tf.keras.layers.Dense(HORIZON)
#])

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15), return_sequences=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(32),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    dX_train.append(train_inputs['X'])
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
for i in range(1,28):
    FD_predictions = FULL_LSTMIMO.predict(dX_test[i-1]['X'])
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i-1], HORIZON, dX_scaler[i-1])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/global/stacked_LSTM.csv')

100% |########################################################################|


Epoch 1/100
12661/12661 [==============================] - 565s 45ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 2/100
12661/12661 [==============================] - 572s 45ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 3/100
 2294/12661 [====>.........................] - ETA: 7:21 - loss: 0.0063 - mean_squared_error: 0.0063

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



12661/12661 [==============================] - 469s 37ms/step - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 4/100
 4430/12661 [=========>....................] - ETA: 3:31 - loss: 0.0062 - mean_squared_error: 0.0062

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

 7645/12661 [=================>............] - ETA: 3:22 - loss: 0.0060 - mean_squared_error: 0.0060

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 8325/12661 [==================>...........] - ETA: 2:53 - loss: 0.0060 - mean_squared_error: 0.0060

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



12661/12661 [==============================] - 538s 42ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 10/100
12661/12661 [==============================] - 542s 43ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 11/100
12661/12661 [==============================] - 542s 43ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 12/100
12661/12661 [==============================] - 549s 43ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 13/100
12661/12661 [==============================] - 553s 44ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 14/100
12661/12661 [==============================] - 545s 43ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0086 - val_mean_squared_e

In [ ]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    dX_train.append(train_inputs['X'])
    dT_train.append(train_inputs['target'])
    dn_test.append(test_inputs.dataframe)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/fulldata_LSTM.csv')

In [24]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
ConditionalRNN = Sequential(layers=[ConditionalRNN(32, cell='LSTM'),
                                    Dense(HORIZON)])

In [29]:
ConditionalRNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


ValueError: You tried to call `count_params` on conditional_rnn_2, but the layer isn't built. You can build it manually via: `conditional_rnn_2.build(batch_input_shape)`.

In [28]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
# ConditionalRNN with the functional API.
#i = Input(shape=[HORIZON, 5], name='time-series')
#c1 = Input(shape=[8], name='one-hot')
#c2 = Input(shape=[2], name='lags-as-features')
# add the condition tensor here.
#x = ConditionalRNN(NUM_CELLS, cell='LSTM', name='cond_rnn_0')([i, c1, c2])
# Dense layer to output
#x = Dense(HORIZON)(x)
#ConditionalRNN = Model(inputs=[i, c1, c2], outputs=[x])

ConditionalRNN = Sequential(layers=[ConditionalRNN(NUM_CELLS, cell='LSTM'),
                                    Dense(HORIZON)])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(1,11))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,27)):
    filename = '../data/Columbia_clean/Residential_'+str(i+1)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        history = ConditionalRNN.fit([train_inputs['X'],train_c1,train_c2], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        #save_model(ConditionalRNN, i)
        predictions = ConditionalRNN.predict([test_inputs['X'],test_c1,test_c2])
        eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/ConditionalRNN10_sequential.csv')

Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by pass

KeyboardInterrupt: 

In [27]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 5)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(1,11))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,27)):
    filename = '../data/Columbia_clean/Residential_'+str(i+1)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, _, _, _, _, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    for j in range(1,11):
        history = LSTMIMO.fit(train_inputs['X'], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = LSTMIMO.predict(test_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/LSTM_MIMO10.csv')

Epoch 1/100
 76/114 [===================>..........] - ETA: 0s - loss: 0.0164 - mean_squared_error: 0.0164

KeyboardInterrupt: 

In [2]:
def train_test_split(df, n_test):
    test_df = df.copy()[-(n_test+23):]
    train_df = df.copy()[:-(len(test_df)-23)]
    return train_df, test_df

In [3]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [4]:
def MIMO_fulldata_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler